**UAS PBA KELAS A**

ANGGOTA KELOMPOK :

1.   Wisnu Wardana   (D1042181001)
2.   Wildan Akmal    (D1042181004)
3.   Renggi Rezki Pratama (D1042181018)




In [ ]:
cd drive/MyDrive/UAS_PBA_Wisnu Wardana

/content/drive/MyDrive/UAS_PBA_Wisnu Wardana


In [ ]:
import random
import json
import pickle
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
intents = json.loads(open('intents.json').read())

In [ ]:
#####Training###

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [ ]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [ ]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel.h5', hist)
print("Done")

####Training Selesai###

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 2/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 3/200
5/5 [==============================] - 0s 2ms/step - loss: 0.0338 - accuracy: 1.0000
Epoch 4/200
5/5 [==============================] - 0s 2ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 5/200
5/5 [==============================] - 0s 2ms/step - loss: 0.0231 - accuracy: 1.0000
Epoch 6/200
5/5 [==============================] - 0s 2ms/step - loss: 0.0355 - accuracy: 1.0000
Epoch 7/200
5/5 [==============================] - 0s 2ms/step - loss: 0.0095 - accuracy: 1.0000
Epoch 8/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0939 - accuracy: 0.9565
Epoch 9/200
5/5 [==============================] - 0s 3ms/step - loss: 0.0114 - accuracy: 1.0000
Epoch 10/200
5/5 [==============================] - 0s 2ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 11/200
5/5 [===========

In [ ]:
####Chatbot#####

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbotmodel.h5')

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [ ]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [ ]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

In [ ]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
print("Go! bot is running!!!")

while True:
    message = input("Masukkan Pesan dan Enter : ")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print("Respon : " +(res))
    break



Go! bot is running!!!
Masukkan Pesan dan Enter : claim bonus
Respon : setiap pengisian pulsa akan mendapatkan bonus berupa kuota internet!
